In [2]:
import tensorflow as tf

# Importing matplotlib to plot images.
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import pandas as pd
# Importing SK-learn to calculate precision and recall
import sklearn
import sklearn.metrics

# Used for graph export
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras.models import load_model
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Dense, Conv2D,MaxPool2D,Dropout,Flatten,Concatenate
from keras.optimizers import RMSprop,Adam

tf.__version__

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


'1.5.0'

## Reading the training set pickle file

In [2]:
train_set = pd.read_pickle('train_set.pkl')
train_set = train_set.sample(frac=1)
print(train_set.shape)

(292634, 8)


## Reading the validation set pickle file

In [3]:
validation_set = pd.read_pickle('test_set.pkl')
validation_set = validation_set.sample(frac=1)
print(validation_set.shape)

(53399, 8)


## Getting values and normalizing the data

In [4]:
y_train = train_set[['x_curr', 'y_curr']].values
sub_input_train = train_set[['x_centroid','y_centroid']].values
y_test = validation_set[['x_curr', 'y_curr']].values
mob_test = validation_set[['x_actual','y_actual']].values
mob_val_test = validation_set[['x_centroid','y_centroid']].values

In [5]:
#Normalizing the train image data

img_rows, img_cols = 27, 15
x_train = []
[x_train.append(np.reshape(a,(img_rows,img_cols, 1))) for a in train_set.aray2]
x_train = np.asarray(x_train)
x_train = x_train.astype(np.float64)
x_train /= 255

In [6]:
#Normalizing the test image data
img_rows, img_cols = 27, 15
x_test = []
[x_test.append(np.reshape(a,(img_rows,img_cols, 1))) for a in validation_set.aray2]
x_test = np.asarray(x_test)
x_test = x_test.astype(np.float64)
x_test /= 255

In [7]:
del train_set
del validation_set

## Training GPU Configuration

In [8]:
# If GPU is not available: 
# GPU_USE = '/cpu:0'
# config = tf.ConfigProto(device_count = {"GPU": 0})


# If GPU is available: 
config = tf.ConfigProto()
config.log_device_placement = True
config.allow_soft_placement = True
config.gpu_options.allocator_type = 'BFC'

# Limit the maximum memory used
config.gpu_options.per_process_gpu_memory_fraction = 0.10

# set session config
tf.keras.backend.set_session(tf.Session(config=config))

In [9]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

## Hyperparameters and helper functions

In [10]:
#hyper parameters
batch_size = 64
epochs = 100
optimizer = RMSprop(lr=0.001)

## Loss function for the model

In [4]:
def distance(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean((tf.square(y_pred[:,0]-y_true[:,0])+tf.square(y_pred[:,1]-y_true[:,1]))))

def testmodel(epoch, logs):
    pred_y = model.predict(
        [x_test,mob_val_test],
        batch_size=batch_size
    )
    true_y = y_test

    eX = np.abs(true_y[:,0] - pred_y[:,0]) 
    eY = np.abs(true_y[:,1] - pred_y[:,1])
    ListError = eX + eY
    ListErrorX = eX
    ListErrorY = eY

    diff_true = true_y[:,0] + true_y[:,1]
    diff_pred = pred_y[:,0] + pred_y[:,1]
    RMSE = np.sqrt(np.mean(np.square(np.subtract(diff_pred, diff_true))))
    RMSEX = np.sqrt(np.mean(np.square(np.subtract(pred_y[:,0], true_y[:,0]))))
    RMSEY   = np.sqrt(np.mean(np.square(np.subtract(pred_y[:,1], true_y[:,1]))))

    print ('\nRMSE: %.2f RMSE-X: %.2f RMSE-Y: %.2f e: %.2f E-X: %.2f E-Y: %.2f std: %.2f std-X: %.2f std-Y: %.2f' %(
              RMSE, RMSEX, RMSEY,
                ListError.mean(), ListErrorX.mean(), ListErrorY.mean(),
                ListError.std(), ListErrorX.std(), ListErrorY.std(),))

testcallback = tf.keras.callbacks.LambdaCallback(on_epoch_end=testmodel)

## Model structure 

In [12]:
main_input = Input(shape=input_shape, dtype='float64', name='main_input')
conv2d = Conv2D(8, kernel_size=(3, 3),
                                 activation='relu',
                                 input_shape=input_shape)(main_input)
maxpool = MaxPool2D(pool_size=(2, 2),
                                 padding='same')(conv2d)
dropout = Dropout(rate=0.5)(maxpool)
conv2d_1 = Conv2D(8, kernel_size=(3, 3),
                                 activation='relu')(dropout)
maxpool_1 = MaxPool2D(pool_size=(2, 2),
                                 padding='same')(conv2d_1)
dropout_1 = Dropout(rate=0.5)(maxpool_1)
flatten = Flatten()(dropout_1)
dense = Dense(8,activation='relu',name='dense')(flatten)
sub_input = Input(shape=(2,), dtype='float32', name='sub_input')
densee = Dense(2,activation='relu')(sub_input)
concat = Concatenate(axis=1)
x = concat([dense,densee])
dense1 = Dense(16, activation='relu')(x)
dense2 = Dense(8, activation='relu')(dense1)
out = Dense(2, activation='linear')(dense2)
model = Model(inputs=[main_input, sub_input], outputs=out)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 27, 15, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 25, 13, 8)    80          main_input[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 13, 7, 8)     0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 13, 7, 8)     0           max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
conv2d_2 (

In [ ]:
model.compile(optimizer=optimizer, loss=distance,
              metrics=['mae'])

history = model.fit([x_train, sub_input_train], y_train,
          epochs=epochs, batch_size=batch_size,
                    verbose=1,validation_data=([x_test,mob_val_test],y_test),
                    shuffle=True,
                    callbacks=[testcallback])

In [ ]:
#model prediction
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
predict = model.predict([x_test,mob_test])

## Error comparison between model and mobile touch points with cross hair mid point

In [76]:
def dist(y_pred,y_true):
    return np.mean(np.sqrt(np.square(np.abs(y_pred[:,0] - y_true[:,0]))+ np.square(np.abs(y_pred[:,1] - y_true[:,1]))))

print(predict[0])
print(y_test[0])
print(mob_test[0])
predict = predict.astype(np.float64)
model_error = (dist(y_test,predict))
mobile_error = dist(y_test,mob_test)
print(model_error)
print(mobile_error)

[341.8997 784.653 ]
[323. 779.]
[358. 834.]
41.23267151547041
50.676487295081365
